### CNN for DDoS intrusion detection

In [1]:
import numpy as np
import pandas as pd

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns; sns.set()

from keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.models import Sequential

from keras import callbacks
from keras.callbacks import CSVLogger

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
import tensorflow as tf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


Read and prepare dataset

In [2]:
np.random.seed(667)  # randomize

testdata = pd.read_csv("DDoS2007_spounged.csv", header=None)


#X = traindata.iloc[:,0:42]
#Y = traindata.iloc[:,0]
y_ini = testdata.iloc[:,0]
X_ini = testdata.iloc[:,1:42]

scaler = Normalizer().fit(X_ini)
X_tras = scaler.transform(X_ini) #X Transformed

# reshape input to be [samples, time steps, features]
X = np.reshape(X_tras, (X_tras.shape[0],X_tras.shape[1],1))
y = np.array(y_ini)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

CNN Structure

In [3]:
cnn = Sequential()
cnn.add(Conv1D(filters=16, kernel_size=3, strides=1, padding="same",activation="relu",input_shape=(41, 1)))
cnn.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
cnn.add(MaxPooling1D(pool_size=(2)))
cnn.add(Flatten())
cnn.add(Dense(64, activation="relu"))
cnn.add(Dropout(0.5))
cnn.add(Dense(1, activation="sigmoid"))

cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])

Training (FIT)

In [4]:
cnn.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test))

Train on 700000 samples, validate on 300001 samples
Epoch 1/15


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv1d_1/convolution/Conv2D}}]]
	 [[{{node metrics/acc/Mean_1}}]]

In [ ]:


# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('BRNN Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower right')
plt.savefig('BRNN Model Accuracy.png')
plt.show()



Results

In [ ]:
y_prob = cnn.predict(X_test)
y_pred = (y_prob > 0.5).astype(np.int)
np.savetxt("cnn.txt", y_pred)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred , average="binary")
precision = precision_score(y_test, y_pred , average="binary")
f1 = f1_score(y_test, y_pred, average="binary")

print("confusion matrix")
print("----------------------------------------------")
print("accuracy %.4f" %accuracy)
print("recall %.4f" %recall)
print("precision %.4f" %precision)
print("f1score %.4f" %f1)
cm = metrics.confusion_matrix(y_test, y_pred)
print("confusion matrix", cm)
print("----------------------------------------------")